In [ ]:
import RPi.GPIO as GPIO
from time import sleep
import paho.mqtt.client as mqtt
import grove_rgb_lcd
import grovepi
GPIO.setmode(GPIO.BCM)

in_topic = '/lego/in/'
out_topic = '/lego/out/'

mqtt_host = 'iot.pue.es'
mqtt_port = 1883


### Define Objects
##Mqtt client object
client = mqtt.Client()
#client.tls_set("ca.pem")
#client.tls_set("ca.pem", "rpi11.pem", "rpi11.key")

##in_topics
door_topic = in_topic+'door'
led_topic = in_topic+'led'

##out_topics
doorbell_topic = out_topic+'doorbell'
temp_topic = out_topic+'temp'
hum_topic = out_topic+'hum'
ldr_topic = out_topic+'ldr'


led = 8
buttons = (4,17)
out = dict()
grovepi.pinMode(led, 'OUTPUT')

def when_rising(channel):
    global out
    
    out[channel] = not out[channel]
    if channel == buttons[0]:
        client.publish('/lego/in/led',int(out[channel]))
    if channel == buttons[1]:
        client.publish('/lego/in/door',int(out[channel]))

sensors=dict()
sensors['temp']=0
sensors['hum']=0
sensors['ldr']=0

def on_message(client, userdata, msg):
    global sensors
    
    try:
        
        if msg.topic == doorbell_topic:
            
            if int(msg.payload) == 1:
                grovepi.digitalWrite(led, 1)
            elif int(msg.payload) == 0:
                grovepi.digitalWrite(led, 0)
        elif msg.topic == hum_topic or msg.topic == temp_topic or msg.topic == ldr_topic:
            sensor=msg.topic.split("/")[-1]
            sensors[sensor] = float(msg.payload)
            
    except:
        pass
        
client.connect(mqtt_host, mqtt_port)
client.subscribe(out_topic+"#")
client.on_message = on_message
client.loop_start()

        
for b in buttons:
    GPIO.setup(b, GPIO.IN,pull_up_down=GPIO.PUD_DOWN)
    out[b] = False
    GPIO.add_event_detect(b,GPIO.RISING,bouncetime=500)
    GPIO.add_event_callback(b,when_rising) 


try:
    while True:    
        grove_rgb_lcd.setRGB(255,255,255)
        text = 'T:{}C H:{}%\nLDR: {}'.format(sensors['temp'],sensors['hum'],int(sensors['ldr']))
        try:
            grove_rgb_lcd.setText_norefresh(text)
        except IOError:
            pass
    
        sleep(1)
finally:
    for b in buttons:
        GPIO.remove_event_detect(b)
    client.loop_stop()
    client.disconnect()
    